## Line total files per month

In [ ]:
import pandas as pd 
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import seaborn as sns


In [ ]:
df_street = pd.read_csv('data/metro_street.csv')

In [ ]:
df_street.dropna(subset=['Borough'], inplace=True)

In [ ]:
#districts_to_include = df_street[df_street['LSOA name'].str.startswith("City")]['LSOA name'].unique()

In [ ]:
#df_street_cleaned = df_street[df_street['LSOA name'].isin(districts_to_include)]

In [ ]:
df_street_cleaned = df_street.drop(columns=['Last outcome category'])
df_street_cleaned = df_street_cleaned[df_street_cleaned['Crime type'] != 'Possession of weapons']

In [ ]:
df_street_cleaned['Month'] = pd.to_datetime(df_street_cleaned['Month'])

In [ ]:
monthly_dict = {}

for month in df_street_cleaned['Month'].dt.month_name().unique():
    df_month = df_street_cleaned[df_street_cleaned['Month'].dt.month_name() == month]
    monthly_dict[month] = {}
    for lsoa in df_month['Borough'].unique():
        df_lsoa = df_month[df_month['Borough'] == lsoa]
        crimes_per_year = df_lsoa.groupby(df_lsoa['Month'].dt.year).count().Month 
        X = crimes_per_year.index.values.reshape(-1, 1)
        y = crimes_per_year

        model = linear_model.LinearRegression().fit(X, y)
        monthly_dict[month][lsoa] = model.coef_




    


In [ ]:
crime_index = pd.read_csv('updated_crime_index.csv')

In [ ]:
crime_dict = pd.Series(crime_index['CCHI Score'].values,index=crime_index['SUB_GROUP']).to_dict()

In [ ]:
crime_dict

In [ ]:
df_street_cleaned['Crime Score'] = df_street_cleaned['Crime type'].replace(crime_dict)	

In [ ]:
coef_list = []
for index, row in df_street_cleaned.iterrows():
    coef = monthly_dict[row['Month'].month_name()][row['Borough']]
    coef_list.append(coef)
df_street_cleaned['Coefficient'] = coef_list

In [ ]:
df_street_cleaned['Coefficient'] = df_street_cleaned['Coefficient'].astype('float')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_street_cleaned[['Crime Score', 'Coefficient']] = scaler.fit_transform(df_street_cleaned[['Crime Score', 'Coefficient']])

In [ ]:
df_street_cleaned['Final Coefficient'] = df_street_cleaned['Crime Score'] * df_street_cleaned['Coefficient']

In [ ]:
scaler = MinMaxScaler()
df_street_cleaned['Final Coefficient'] = scaler.fit_transform(df_street_cleaned['Final Coefficient'].values.reshape(-1, 1))

In [ ]:
df_street_cleaned = pd.read_csv('final_street.csv')

In [ ]:
df_street_cleaned['Month'] = pd.to_datetime(df_street_cleaned['Month'])

In [ ]:
data = df_street_cleaned[(df_street_cleaned['Month'].dt.month_name() == 'January')| (df_street_cleaned['Month'].dt.year == 2019)]
data = data.loc[:, ['Longitude', 'Latitude', 'Final Coefficient']]
X = data.loc[:, ['Longitude', 'Latitude']].values
Y = data['Final Coefficient'].values

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, n_init=10, random_state=0, max_iter=1000)

wt_kmeansclus = kmeans.fit(X,sample_weight = Y)
predicted_kmeans = kmeans.predict(X, sample_weight = Y)


In [ ]:
wt_kmeansclus.cluster_centers_[:,0]

In [ ]:
import plotly.express as px

In [ ]:
px.set_mapbox_access_token(open("mapbox_token").read())
nicemap = px.scatter_mapbox( lat = X[:, 1], lon = X[:, 0], color = predicted_kmeans,
size_max=15, zoom=10)
nicemap.show()